#소프트맥스 회귀

multi-class classifier에 사용된다. Class의 값이 수치가 아닌 이름, 명칭 등일 때 사용한다.

각 class의 분포는 동일하다는 가정 하에 무작위적으로 각 class를 정의하므로 multi-class에 적용하기 적합하다. 하지만, class간의 유사도를 계산할 수 없기 때문에 한계가 존재한다.

#소프트맥스 회귀

로지스틱 회귀처럼, 각 class가 나올 확률들의 합은 1이 된다는 아이디어를 사용한다.

가설: H(X) = softmax(WX+B)

i번째 클래스가 정답일 확률을 p(i)라고 할 때, p(i) = e^z(i)/sigma(e^z)라고 정의한다. 따라서, k개의 class에 대해서는 출력값이 [p(1), p(2), p(3), ... p(k)]이다.

#소프트맥스에 대한 질문

오차는 어떻게 계산할 것인가. 예측값 [p(1), p(2), p(3), ... p(k)]가 있다고 할 때, 오차를 구하기 위해서는 예측값과 실제값을 비교해야 한다. 하지만 실제값은 확률값이 아니라 어떤 특정 class일텐데 어떻게 확률과 class 값을 비교하지?

소프트맥스는 실제값을 one hot vector로 encoding한다. p1, p2, p3 중에 2번째가 실제값이라면, 값을 [0, 1, 0]으로 encoding한다. 따라서, 예측값이 [0.1, 0.6, 0.3]으로 나왔다면, [0.1, 0.6, 0.3]와 [0, 1, 0]을 비교하여 최적화를 한다.

#실제 데이터로 해보기

붓꽃 품종 분류를 한다고 할 때, data 수를 5개, feature는 4개라고 하자. 총 class수는 3개라고 하자. 예측값은 5x3, 입력값은 5x4이다/ 따라서 W는 4x3임을 알 수 있다.

소프트맥스 회귀에서 비용 함수는 크로스 엔트로피 함수를 사용한다.

cost(W,b) = -sigma(y*log(p(i))로 정의한다. 실제 값이 1이고 p(i)가 1일 때, cost가 0. p(i)가 0일 때, cost는 무한이 된다.

#소프트맥스 회귀의 비용 함수 구현하기


In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

In [2]:
z = torch.FloatTensor([1, 2, 3])
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


각 class가 나올 확률의 총 합이 1인지

In [3]:
hypothesis.sum()

tensor(1.)

5x3 입력값에 대해 각 클래스가 나올 확률의 예측값.

In [5]:
z = torch.rand(5,3,requires_grad=True)
hypothesis = F.softmax(z,dim=1)
print(hypothesis)

tensor([[0.3156, 0.3549, 0.3295],
        [0.4365, 0.3294, 0.2341],
        [0.2999, 0.3731, 0.3270],
        [0.2643, 0.3177, 0.4180],
        [0.3990, 0.1865, 0.4145]], grad_fn=<SoftmaxBackward0>)


레이블 설정

In [12]:
y = torch.randint(3, (5,)).long()
print(y)

tensor([0, 0, 0, 2, 1])


In [13]:
#각 레이블에 대한 one hot encoding
y_onehot = torch.zeros_like(hypothesis)
y_onehot.scatter_(1, y.unsqueeze(1), 1)
print(y_onehot)

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.]])


In [14]:
cost = (y_onehot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.1476, grad_fn=<MeanBackward0>)


소프트맥스 비용 함수 high level로 구현


In [15]:
#1. F.softmax() + torch.log() = F.log_softmax()

F.log_softmax(z, dim=1)

tensor([[-1.1533, -1.0358, -1.1103],
        [-0.8289, -1.1104, -1.4522],
        [-1.2042, -0.9860, -1.1178],
        [-1.3307, -1.1465, -0.8724],
        [-0.9188, -1.6794, -0.8806]], grad_fn=<LogSoftmaxBackward0>)

In [16]:
F.log_softmax(z, dim=1)

tensor([[-1.1533, -1.0358, -1.1103],
        [-0.8289, -1.1104, -1.4522],
        [-1.2042, -0.9860, -1.1178],
        [-1.3307, -1.1465, -0.8724],
        [-0.9188, -1.6794, -0.8806]], grad_fn=<LogSoftmaxBackward0>)

In [17]:
#2. -y*log(y^)을 구현 쉽게
#cost = (y_onehot * -torch.log(hypothesis)).sum(dim=1).mean()
(y_onehot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.1476, grad_fn=<MeanBackward0>)

In [18]:
#대신
F.nll_loss(F.log_softmax(z,dim=1), y) #nll -> negative log likelihood
#one hot vector를 적용하지 않아도 된다.


tensor(1.1476, grad_fn=<NllLossBackward0>)

In [19]:
#더 간단하게
F.cross_entropy(z,y)

tensor(1.1476, grad_fn=<NllLossBackward0>)